In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random

import gc
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from PIL import Image
import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = 9000000000000

import tensorflow_datasets as tfds
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, Callback


import openslide
from openslide import OpenSlide

# remove cap for image reading
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2 # import after setting OPENCV_IO_MAX_IMAGE_PIXELS

# Jobs:

+ Learn https://towardsdatascience.com/using-shap-values-to-explain-how-your-machine-learning-model-works-732b3f40e137?gi=578a522915

+ Check Domino Data lab

# To-do:

 + Create a new convergence system:
 > The system should have a log(n) time complexity
 + Define the boundaries of each iamge.  Weight if it is better to have a fixed initial window size or make the intial window dependent on the image size
 + Edit the loss scoring method and let it score higher when you find nucleoids + redcells + fat tissue
 

In [2]:
train = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')
test = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [3]:
# add pic file_path
train['file_path'] = train['image_id'].apply(lambda x: '../input/mayo-clinic-strip-ai/train/' + x + '.tif')
train.head()

,image_id,center_id,patient_id,image_num,label,file_path
0,006388_0,11,006388,0,CE,../input/mayo-clinic-strip-ai/train/006388_0.tif
1,008e5c_0,11,008e5c,0,CE,../input/mayo-clinic-strip-ai/train/008e5c_0.tif
2,00c058_0,11,00c058,0,LAA,../input/mayo-clinic-strip-ai/train/00c058_0.tif
3,01adc5_0,11,01adc5,0,LAA,../input/mayo-clinic-strip-ai/train/01adc5_0.tif
4,026c97_0,4,026c97,0,CE,../input/mayo-clinic-strip-ai/train/026c97_0.tif


In [4]:
joo = ['09644e_0', '008e5c_0', '00c058_0', '09644e_2', '09644e_1']
train[train['image_id'].isin(joo)]

,image_id,center_id,patient_id,image_num,label,file_path
1,008e5c_0,11,008e5c,0,CE,../input/mayo-clinic-strip-ai/train/008e5c_0.tif
2,00c058_0,11,00c058,0,LAA,../input/mayo-clinic-strip-ai/train/00c058_0.tif
25,09644e_0,10,09644e,0,CE,../input/mayo-clinic-strip-ai/train/09644e_0.tif
26,09644e_1,10,09644e,1,CE,../input/mayo-clinic-strip-ai/train/09644e_1.tif
27,09644e_2,10,09644e,2,CE,../input/mayo-clinic-strip-ai/train/09644e_2.tif


In [5]:
def periodic(val):
    wave = np.cos(2 * np.pi * 1/800 * val + 1/8 * np.pi) + 2
    return wave

In [6]:
def get_loss(region):
    mat = np.array(region)/255.
    # treshold of squares colors
    tresh = 1.2
    # treshold of the thickness of the edge:
    edge = 0.5
    # number of good choices
    good = 0
    N = mat.shape
    # resolution of the region (should be kept fixed)
    x_n, y_n = N[0], N[1]
    n_l = (np.ceil(x_n * 0.01), np.ceil(y_n * 0.01)) # check for 1% of the total image
    good2 = False
    # start
    x = 0
    y = 0
    # step 
    step = (x_n/n_l[0], y_n/n_l[1])
    for i in range(int(n_l[0])):
        y = 0
        pos_x = i * step[0]
        for j in range(int(n_l[1])):
            pos_y = j * step[1]
            # pick different values for the coordinates
            pixel = mat[x, y]
            
            # calculate the cost: check if the pixel in position x, y is not white
            res = np.sqrt(pixel[1]**2 + pixel[2]**2) + 0.1/pixel[0]
            
            if res <= tresh:
                good2 = True
                good += 1
            wave = periodic(pos_y)
            y += int(step[1] * 0.5 * wave)
        wave = periodic(pos_x)
        x += int(step[0] * 0.5 * (wave))
    # return the fraction of good pixels in the region
    res = good/max(n_l)
    if res >= 0.2:
        return res
    return 0

In [ ]:
def check_cross(slide, point, window_size, tresh):
    m_loss = [0, 0, 0, 0]
    # check right
    point[0] += window_size[0]
    window = slide.read_region(point, 0, window_size)
    m_loss[1] = get_loss(window)

    # check left
    point[0] = start_point[0]
    point[0] -= window_size[0]
    window = slide.read_region(point, 0, window_size)
    m_loss[0] = get_loss(window)
    
    if m_loss[1] <= tresh and m_loss[0] <= tresh:
        return m_loss
    
    # check bottom
    point[1] += window_size[1]
    window = slide.read_region(point, 0, window_size)
    m_loss[2] = get_loss(window)

    # check top
    point[1] = start_point[1]
    point[1] -= window_size[1]
    window = slide.read_region(point, 0, window_size)
    m_loss[3] = get_loss(window)
    
    return m_loss

In [7]:
# return the ammount of blood (in fraction) in a 5k x 5k region
def check_window(slide, start_point, good_tresh, window_size, p_step = 1, deep = 0):
    point = start_point.copy()
    window = slide.read_region(point, 0, window_size)
    loss = get_loss(window)
    m_loss = [0, 0, 0, 0]
    
    if loss >= good_tresh:
        m_loss = check_cross(slide, point, window_size, good_tresh)
        
        # caso base
        loss = min(m_loss)
        if loss >= good_tresh or max(m_loss) <= good_tresh:
            print(f"Retornando |{loss}|")
            return loss, point
        
        if m_loss[0] <= good_tresh and m_loss[1] >= good_tresh:
            # go right
            if m_loss[2] <= good_tresh and m_loss[3] >= good_tresh:
                # go up
                point[0] += window_size[0] * p_step
                point[1] += window_size[1] * p_step
                return check_window(slide, point, good_tresh, window_size, p_step, deep + 1)
                
            elif m_loss[2] >= good_tresh and m_loss[3] <= good_tresh:
                # go down
                point[0] += window_size[0] * p_step
                point[1] -= window_size[1] * p_step
                return check_window(slide, point, good_tresh, window_size, p_step, deep + 1)
                
        elif m_loss[0] >= good_tresh and m_loss[1] <= good_tresh:
            # go left
            if m_loss[2] <= good_tresh and m_loss[3] >= good_tresh:
                # go up
                point[0] -= window_size[0] * p_step
                point[1] += window_size[1] * p_step
                return check_window(slide, point, good_tresh, window_size, p_step, deep + 1)
                
            elif m_loss[2] >= good_tresh and m_loss[3] <= good_tresh:
                # go down
                point[0] -= window_size[0] * p_step
                point[1] -= window_size[1] * p_step
                return check_window(slide, point, good_tresh, window_size, p_step, deep + 1)    
    # big if 
    return loss, point
    

In [8]:
def function(path, n_pixels = 3.6e6, window_size = (100, 100)):
    # get width and height of image
    _ = PIL.Image.open(path)
    width, height = _.size
    
    # Number of windows 
    n_windows = int(n_pixels / (window_size[0] * window_size[1]))
    
    # number of pixels between each (window_size) window
    step = ((width - window_size[0] * n_windows) / n_windows , (height - window_size[1] * n_windows) / n_windows)
    start = [500, 500]
    
    for i in range(n_windows):
        # read and score region
        ## the score shoulud be based on the ammount of information present on the window 
        ## + the ammount of info present on the surroundings
        
        start[0] += step[0]
        start[1] += step[1]
        print("Openning slide...")
        slide = OpenSlide(path)
        print("Done")
        check_window(slide, start)
    #  ----------------------------
    
    
    MAX_LOSS = 1
    MAX_REGION = [1,1,1]
    avg = 0
    rows = []
    for i in range(5):
#         print(f"Window of row: {i}")
        for j in range(5):
            max_r, m_region = check_big_window(slide, starting)
            if MAX_LOSS <= max_r:
                MAX_LOSS = max_r
                MAX_REGION = m_region

            starting[0] += 5000
        starting[1] += 5000
        
    return MAX_LOSS, MAX_REGION

In [ ]:
varios = train[train['label'] == 'CE'].iloc[0:2].file_path.to_numpy()
varios = np.concatenate([varios, train[train['label'] == 'LAA'].iloc[0:2].file_path.to_numpy()])
varios

In [ ]:
%%time
sample_train = train[:20]
regions = []
losses = []

varios = train[train['label'] == 'CE'].iloc[0:5].file_path.to_numpy()
varios = np.concatenate([varios, train[train['label'] == 'LAA'].iloc[0:5].file_path.to_numpy()])

im = PIL.Image.open(varios[0])
print(im.size)

# the default max_size of pics is 30_000 x 30_000
for i in varios:
    path = i
#     slide = OpenSlide(path)
    loss, region = function(path)
    print(f"Pic {path}")
    window = slide.read_region(region, 0, (800, 800))
    print(loss, region)
    plt.figure(figsize = (8, 8))
    plt.imshow(window)
    plt.show()
    size = (800, 800)
    x = 500
    y = 500
    print("Done!")

In [ ]:
# directly process the data
def process(path):
    size  = (800, 800)
    slide = OpenSlide(path)
    loss, region = function(slide)
    image = slide.read_region(region, 0, size)
    plt.figure(figsize = (8, 8))
    plt.imshow(image)
    plt.show()
    image = tf.image.resize(image, (512, 512))
    return image

In [ ]:
X = []
for path in tqdm(train['file_path']):
    print(f"Processing {path}")
    image = process(path)
    X.append(image)
    name = path.split("/")[-1]
    tf.keras.utils.save_img(f"/kaggle/working/processed/{name}", image)

In [ ]:
toy = train.copy()
files = []
paths = os.scandir("/kaggle/working/processed")
for part in paths:
    if part.is_file() and (part.name.find("tif") > 0):
        files.append(part.name)
        

toy = toy.set_index('image_id')

for file in files:
    print()
    name = file.split(".")[0]
    label = toy.loc[name, 'label']
    print(name, label)

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * drop ** np.floor((epoch)/epochs_drop)
    return lrate

l_rate = LearningRateScheduler(step_decay)
earstop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5)

In [ ]:
shapes = (512, 512, 4)

input_layer = layers.Input(name = 'input', shape = shapes)
conv_1 = layers.Conv2D(filters=32, kernel_size = (3,3), strides =2, padding = 'same', activation = 'relu', input_shape = shapes, name = 'conv_1')(input_layer)
conv_2 = layers.Conv2D(filters=64, kernel_size = (3,3), strides =2, padding = 'same', activation = 'relu', name = 'conv_2')(conv_1)
conv_3 = layers.Conv2D(filters=32, kernel_size = (3,3), strides =2, padding = 'same', activation = 'relu', name = 'conv_3')(conv_2)
flat = layers.Flatten()(conv_3)
h1 = layers.Dense(128, activation = 'relu', name = 'h1')(flat)
drop = layers.Dropout(0.25)(h1)
output = layers.Dense(1, activation = 'sigmoid', name = 'prediction')(h1)

model = tf.keras.Model(input_layer, output)

model.compile(optimizer = 'adam', 
                loss = tf.keras.losses.BinaryCrossentropy(), 
                 metrics = ['accuracy', 'mse', 'mape']
             )

tf.keras.utils.plot_model(model)

In [ ]:
vocab = train['label'].unique().tolist()
Y = train['label'].apply(lambda x: vocab.index(x))

In [ ]:
ret = pd.DataFrame((X, Y)).transpose()
ret.columns = ['input', 'label']
ret.to_csv('processed.csv', index = False, header = None)

In [ ]:

dataset = pd.read_csv('processed.csv')
train, test = train_test_split(dataset, test_size = 0.2)
train, val = train_test_split(dataset, test_size = 0.2)

train = tf.data.Dataset.from_tensor_slices((dict(train))).batch(BATCH_SIZE)
val = tf.data.Dataset.from_tensor_slices((dict(val))).batch(BATCH_SIZE)
test = tf.data.Dataset.from_tensor_slices((dict(test))).batch(1)
dataset.head()

In [ ]:
for i in train.take(1):
    print(i)

In [ ]:
EPOCHS = 20
STEPS_X_EPOCH = 2
history = model.fit(train, validation_data = val, epochs = EPOCHS, callbacks = [l_rate, earstop])

In [ ]:
pred = model.predict(test_df)
pred

In [ ]:
y = y_test.to_numpy()
y_ = []
for i in pred:
    i_r = i.round()
    if i_r >= 1:
        y_.append(1)
    else:
        y_.append(0)

print(f" Accuracy: {1 - sum(abs(y - y_))/len(y_)}")

In [ ]:
X

In [ ]:
for __ in test_df.take(1):
    print(__)